Code has been generalised so that you can upload your own datasets and paths to your pipeline


In [ ]:
!pip install tensorflow
!pip install tf_slim
!pip install opencv-python-headless
!pip install lxml
!pip install pillow
!pip install numpy
!pip install matplotlib
!pip install Cython
!pip install contextlib2
!pip install jupyter
!pip install pandas
!pip install tf-models-official

In [ ]:
import numpy as np
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.builders import model_builder
from object_detection.utils import visualization_utils as viz_utils
import cv2

In [ ]:
# Path to the pipeline config file
pipeline_config = 'path/to/your/pipeline.config'

# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(pipeline_config)
model_config = configs['model']
detection_model = model_builder.build(model_config=model_config, is_training=True)


In [ ]:
from object_detection.utils import dataset_util
from object_detection.protos import input_reader_pb2
from google.protobuf import text_format

# Load the TFRecord dataset
train_record_path = 'path/to/train.record'
val_record_path = 'path/to/val.record'
label_map_path = 'path/to/label_map.pbtxt'

# Prepare the dataset
def create_tf_example(example):
    # Create a TFExample based on your data
    pass

def load_dataset(record_path):
    raw_dataset = tf.data.TFRecordDataset(record_path)
    parsed_dataset = raw_dataset.map(create_tf_example)
    return parsed_dataset

train_dataset = load_dataset(train_record_path)
val_dataset = load_dataset(val_record_path)


In [ ]:
num_epochs = 10
steps_per_epoch = 100
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

@tf.function
def train_step(image_tensors, groundtruth_boxes_list, groundtruth_classes_list):
    with tf.GradientTape() as tape:
        preprocessed_images = tf.cast(image_tensors, dtype=tf.float32)
        predictions = detection_model(preprocessed_images, training=True)
        losses = detection_model.loss(predictions, groundtruth_boxes_list, groundtruth_classes_list)

    gradients = tape.gradient(losses['Loss/total_loss'], detection_model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, detection_model.trainable_variables))
    return losses

for epoch in range(num_epochs):
    for step, (images, boxes, classes) in enumerate(train_dataset.take(steps_per_epoch)):
        losses = train_step(images, boxes, classes)
        if step % 10 == 0:
            print(f'Epoch {epoch + 1}, Step {step}, Loss: {losses["Loss/total_loss"].numpy()}')
